## LHS 712 Assg 3

# **CRF Model**

In [4]:
# Read in data

def read_file(f):
    # Read all the lines in
    data = open(f,'r').readlines()[1:]
    # Split on tab and save row_id
    row_id = [i.split('\t')[0].strip() for i in data]
    # Split on tab then split words by space
    data = [i.split('\t')[1].strip().split(' ') for i in data]
    return row_id,data

In [5]:
row_id_text, texts = read_file('REVIEW_TEXT.txt')
row_id_tags, tags = read_file('REVIEW_LABELSEQ.txt')

# row_id_text should match exactly with row_id_tags
# len(texts) and len(tags) should match exactly

FileNotFoundError: [Errno 2] No such file or directory: 'REVIEW_TEXT.txt'

In [3]:
row_id_text

NameError: name 'row_id_text' is not defined

In [141]:
# Generate features (add features for improvement)
import string

def word2features(word):
    features = {
        'word': word,
        'word.lower()': word.lower(),
        'word.upper()': word.upper(),
        'word[:1]': word[:1],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word[:4]': word[:4],
        'word[-1:]': word[-1:],
        'word[-2:]': word[-2:],
        'word[-3:]': word[-3:],
        'word[-4:]': word[-4:],
        'word.istitle()': word.istitle(),
        'word.ispunctuation()': (word in string.punctuation),
        'len(word)': len(word)
    }
    return features

# run word2features on every word in text
def text2features(text):
    return [word2features(i) for i in text]

In [142]:
x = [text2features(text) for text in texts]
y = tags
# x is now features and y is associated tags

In [143]:
# Splitting Train / Validation Sets

from sklearn.model_selection import train_test_split

x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size = 0.2)

In [144]:
# Running the CRF model

from sklearn_crfsuite import CRF
crf = CRF(algorithm = 'ap')

# Training and Prediction

crf.fit(x_train, y_train)
y_pred = crf.predict(x_validation)

# Generating results

from sklearn_crfsuite.metrics import flat_classification_report
report = flat_classification_report(y_validation, y_pred)
print(report)

              precision    recall  f1-score   support

        B-AE       0.67      0.62      0.65       827
       B-SSI       0.58      0.59      0.58       150
        I-AE       0.72      0.47      0.57      1513
       I-SSI       0.28      0.13      0.18        82
           O       0.91      0.96      0.94     11939

    accuracy                           0.88     14511
   macro avg       0.63      0.55      0.58     14511
weighted avg       0.87      0.88      0.87     14511



/Users/lilaalaka/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


# **LSTM Model**

In [145]:
def read_file(f):
    data = open(f,'r').readlines()[1:]
    row_id = [i.split('\t')[0].strip() for i in data]
    data = [i.split('\t')[1].strip().split(' ') for i in data]
    return row_id,data

In [179]:
row_id_text, texts = read_file('REVIEW_TEXT.txt')
row_id_tags, tags = read_file('REVIEW_LABELSEQ.txt')

In [147]:
from nltk.stem.snowball import SnowballStemmer
snow_stemmer = SnowballStemmer(language='english')

texts_stemmed = []

for text in texts:
    def snow_stem(text):
        stem = [snow_stemmer.stem(word) for word in text]
        return stem
    text = snow_stem(text)
    texts_stemmed.append(text)

texts = texts_stemmed

In [180]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Find all the unique words from the text
# Assign every word a number
# Add special padding word with index 0
unique_words = list(set([j for i in texts for j in i]))
word2idx = {j:i+1 for i,j in enumerate(unique_words)}
word2idx["PAD"] = 0

# Collection of tags with label to index and index to label mapping
unique_tags = list(set([j for i in tags for j in i]))
label2idx = {j:i for i,j in enumerate(unique_tags)}
idx2label = {j:i for i,j in label2idx.items()}

# pad sequences makes sequence lengths the same for NN model
x = [[word2idx[j] for j in i] for i in texts]
x = pad_sequences(maxlen = 25, sequences = x, padding = "post", value = word2idx["PAD"])
y = [[label2idx[j] for j in i] for i in tags]
y = pad_sequences(maxlen = 25, sequences = y, padding = "post", value = label2idx["O"])
y = [to_categorical(i, num_classes = len(unique_tags)) for i in y]
# tf_keras documentation reference
# could use word embedding to convert word to vector instead

In [181]:
# Training and Validation Sets
x_train, x_validation, y_train, y_validation  = train_test_split(x, y, test_size = 0.2)

# Running LSTM Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Activation, Dropout, TimeDistributed
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(input_dim=len(word2idx.keys()),output_dim=20,input_length=25))
model.add(TimeDistributed(Dense(len(label2idx.keys()))))
model.add(Bidirectional(LSTM(units=50,return_sequences=True), merge_mode = 'concat'))
model.add(Dense(len(label2idx.keys()), activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# Reference Tensor Flow documentation 

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    (None, 25, 20)            173680    
                                                                 
 time_distributed_4 (TimeDis  (None, 25, 5)            105       
 tributed)                                                       
                                                                 
 bidirectional_34 (Bidirecti  (None, 25, 100)          22400     
 onal)                                                           
                                                                 
 dense_55 (Dense)            (None, 25, 5)             505       
                                                                 
Total params: 196,690
Trainable params: 196,690
Non-trainable params: 0
_________________________________________________________________


In [182]:
# Training and Prediction

import numpy as np
history = model.fit(x_train,np.array(y_train),batch_size=16,epochs=8,validation_split=0.1)

y_pred = model.predict(x_validation)
y_pred = np.argmax(y_pred, axis=-1)
y_validation = np.argmax(y_validation, -1)
y_pred = [[idx2label[i] for i in row] for row in y_pred]
y_validation = [[idx2label[i] for i in row] for row in y_validation]

Epoch 1/8
214/214 [==============================] - 6s 16ms/step - loss: 0.4542 - accuracy: 0.9044 - val_loss: 0.2773 - val_accuracy: 0.9172
Epoch 2/8
214/214 [==============================] - 3s 12ms/step - loss: 0.2511 - accuracy: 0.9116 - val_loss: 0.2027 - val_accuracy: 0.9265
Epoch 3/8
214/214 [==============================] - 3s 13ms/step - loss: 0.1964 - accuracy: 0.9287 - val_loss: 0.1815 - val_accuracy: 0.9343
Epoch 4/8
214/214 [==============================] - 3s 13ms/step - loss: 0.1590 - accuracy: 0.9435 - val_loss: 0.1771 - val_accuracy: 0.9411
Epoch 5/8
214/214 [==============================] - 3s 14ms/step - loss: 0.1301 - accuracy: 0.9566 - val_loss: 0.1632 - val_accuracy: 0.9472
Epoch 6/8
214/214 [==============================] - 3s 12ms/step - loss: 0.1057 - accuracy: 0.9655 - val_loss: 0.1734 - val_accuracy: 0.9483
Epoch 7/8
214/214 [==============================] - 3s 13ms/step - loss: 0.0890 - accuracy: 0.9701 - val_loss: 0.1588 - val_accuracy: 0.9524
Epoch 

In [183]:
# Results

from sklearn_crfsuite.metrics import flat_classification_report

report = flat_classification_report(y_pred=y_pred, y_true=y_validation)
print(report)

              precision    recall  f1-score   support

        B-AE       0.80      0.53      0.64       695
       B-SSI       0.65      0.58      0.61       161
        I-AE       0.70      0.47      0.56      1329
       I-SSI       0.14      0.02      0.03       108
           O       0.95      0.99      0.97     21432

    accuracy                           0.94     23725
   macro avg       0.65      0.52      0.56     23725
weighted avg       0.93      0.94      0.93     23725



/Users/lilaalaka/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


## **Running LSTM on Test Data**

In [233]:
def read_file(f):
    data = open(f,'r').readlines()[1:]
    row_id = [i.split('\t')[0].strip() for i in data]
    data = [i.split('\t')[1].strip().split(' ') for i in data]
    return row_id,data

row_id_text, texts = read_file('REVIEW_TEXT.txt')
row_id_tags, tags = read_file('REVIEW_LABELSEQ.txt')
row_id_text_test, texts_test = read_file('TEST_REVIEW_TEXT.txt')

In [240]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Find all the unique words from the text
# Assign every word a number
# Add special padding word with index 0
unique_words = list(set([j for i in texts for j in i]))
word2idx = {j:i+1 for i,j in enumerate(unique_words)}
word2idx["PAD"] = 0

# Collection of tags with label to index and index to label mapping
unique_tags = list(set([j for i in tags for j in i]))
label2idx = {j:i for i,j in enumerate(unique_tags)}
idx2label = {j:i for i,j in label2idx.items()}

x = [[word2idx[j] for j in i] for i in texts]
x = pad_sequences(maxlen = 25, sequences = x, padding = "post", value = word2idx["PAD"])
y = [[label2idx[j] for j in i] for i in tags]
y = pad_sequences(maxlen = 25, sequences = y, padding = "post", value = label2idx["O"])
y = [to_categorical(i, num_classes = len(unique_tags)) for i in y]

In [241]:
unique_words_test = list(set([j for i in texts_test for j in i]))
word2idx_test = {j:i+1 for i,j in enumerate(unique_words_test)}
word2idx_test["PAD"] = 0

x_test = [[word2idx_test[j] for j in i] for i in texts_test]
x_test = pad_sequences(maxlen = 25, sequences = x_test, padding = "post", value = word2idx_test["PAD"])

In [242]:
# Running LSTM Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Activation, Dropout, TimeDistributed
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(input_dim=len(word2idx.keys()),output_dim=20,input_length=25))
model.add(TimeDistributed(Dense(len(label2idx.keys()))))
model.add(Bidirectional(LSTM(units=50,return_sequences=True), merge_mode = 'concat'))
model.add(Dense(len(label2idx.keys()), activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# Reference Tensor Flow documentation 

# Fit model
model.fit(x, np.array(y), batch_size=16, epochs=8)

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_40 (Embedding)    (None, 25, 20)            173680    
                                                                 
 time_distributed_10 (TimeDi  (None, 25, 5)            105       
 stributed)                                                      
                                                                 
 bidirectional_40 (Bidirecti  (None, 25, 100)          22400     
 onal)                                                           
                                                                 
 dense_67 (Dense)            (None, 25, 5)             505       
                                                                 
Total params: 196,690
Trainable params: 196,690
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8
297/297 [============================

In [243]:
y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=-1)
#y_validation = np.argmax(y_validation, -1)
y_pred = [[idx2label[i] for i in row] for row in y_pred]
#y_validation = [[idx2label[i] for i in row] for row in y_validation]

In [244]:
# Checking if lengths of predictions and row ids match

print(len(y_pred), len(row_id_text_test))

1259 1259


In [245]:
# Changing BIO labels to string for each sentence

y_pred_temp = []

for sub_list in y_pred:
    sub_list = ' '.join([label for label in sub_list])
    y_pred_temp.append(sub_list)

y_pred = y_pred_temp

In [246]:
# Exporting label results in LABELSEQ.txt format

import pandas as pd

column_names = ['ID','TAGSEQ']
test_labels_df = pd.DataFrame(zip(row_id_text_test,y_pred), columns=column_names)

test_labels_df.to_csv('TEST_LABELSEQ.txt',sep='\t',index=False)

## **Running CRF on Test Data**

In [170]:
def read_file(f):
    data = open(f,'r').readlines()[1:]
    row_id = [i.split('\t')[0].strip() for i in data]
    data = [i.split('\t')[1].strip().split(' ') for i in data]
    return row_id,data

In [171]:
row_id_text, texts = read_file('REVIEW_TEXT.txt')
row_id_tags, tags = read_file('REVIEW_LABELSEQ.txt')
row_id_text_test, texts_test = read_file('TEST_REVIEW_TEXT.txt')

In [172]:
import string
def word2features(word):
    features = {
        'word': word,
        'word.lower()': word.lower(),
        'word.upper()': word.upper(),
        'word[:1]': word[:1],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word[:4]': word[:4],
        'word[-1:]': word[-1:],
        'word[-2:]': word[-2:],
        'word[-3:]': word[-3:],
        'word[-4:]': word[-4:],
        'word.istitle()': word.istitle(),
        'word.ispunctuation()': (word in string.punctuation),
        'len(word)': len(word)
    }
    return features

def text2features(text):
    return [word2features(i) for i in text]

In [173]:
x_train = [text2features(text) for text in texts]
y_train = tags
x_test = [text2features(text) for text in texts_test]

In [174]:
from sklearn_crfsuite import CRF
crf = CRF(algorithm = 'ap')

# Training and Prediction
crf.fit(x_train, y_train)
y_pred = crf.predict(x_test)

In [175]:
# Checking if lengths of predictions and row ids match

print(len(y_pred), len(row_id_text_test))

1259 1259


In [176]:
# Changing BIO labels to string for each sentence

y_pred_temp = []

for sub_list in y_pred:
    sub_list = ' '.join([label for label in sub_list])
    y_pred_temp.append(sub_list)

y_pred = y_pred_temp

In [177]:
# Exporting label results in LABELSEQ.txt format

import pandas as pd

column_names = ['ID','TAGSEQ']
test_labels_df = pd.DataFrame(zip(row_id_text_test,y_pred), columns=column_names)

test_labels_df.to_csv('TEST_LABELSEQ_CRF.txt',sep='\t',index=False)